# Task 0: Getting access to key files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp drive/MyDrive/larsoft_workshop_files/*.py .
!cp drive/MyDrive/larsoft_workshop_files/*.pt .
!cp drive/MyDrive/larsoft_workshop_files/*.gz .
!tar xzf images.tar.gz

# Task 1: Transfer learning for neutral current interaction classification

We'll start by importing all of the Python modules required for this notebook - there are numerous other modules used by supporting code, but we'll ignore those here. We only require direct  access to tools to allow us to copy model state, time our training runs, run various torch code and construct a ResNet.

In [ ]:
import copy, time, torch, torchvision as tv

The files below come from the linked Google Drive and provide custom code to support these tutorials. If you're interested, feel free to look through them and ask questions about what they do, but a detailed understanding shouldn't be necessary for this tutorial

In [ ]:
from data_utils import *
from helpers import *
from model_utils import *
from training_utils import *

The model we'll be using is based on a ResNet18 CNN. PyTorch already have this model defined via torchvision, so we don't need to construct it from scratch ourselves. It's also possible to request that PyTorch download and populate the model with weights determined from extensive training on ImageNet, though we won't do that here, because we'll be using a slightly different set of weights to get started.

The following cell sets up the standard ResNet18 model and provides a summary description. As you can see, even the smallest ResNet has a large number of parameters.

In [ ]:
model = tv.models.resnet18(pretrained=False)
model

A network has already been pretrained on charged current interactions that will form the base network that you will modify and tune throughout the tasks. However, if you tried to load the weights from this model now, it would fail. Why?

In the cell below write code to modify the classifier layers of the ResNet such that it can classify two different types of interaction and display the updated model structure (Hint: arguments to constructors often translate directly to class member variables). Does it look right?

In [ ]:
# TASK - Modify the network architecture here

# Display the model
model

Hopefully you have a correctly structured network now. When you run the cell below you should see a message like

```
<All keys matched successfully>
```

otherwise, you'll likely see an error message below that should provide a clue as to the problem.

You will modify and tune this network throughout the tasks. To get started, you'll find the `model_baseline.pt` file in **shared location directory**, which will need to be uploaded to Google Colab.

We can load this model onto the GPU via the following code.

In [ ]:
filename = 'model_baseline.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(filename, map_location=device))

At this stage you have a two class classification ResNet with weights that have been set based on training on a number of charged current interactions, in particualr CCQE and CCRes. We want to add NCQE and NCRes to the interactions that can be classified, but we don't want to retrain the whole network.

Your next subtask is to freeze the weights we've already loaded, so that they can't be modified. In the cell below, write some code to freeze all of the parameters in our model - you don't need to be subtle about this, we'll further modify the network in a future step that will allow the classification layer to be trained.

A couple of useful methods/variables: classes that inherit from nn.Module (which model does) have a `parameters()` method returning a iterable collection of model parameter objects. Among the member variables of a parameter object is `requires_grad` (in order for a weight to be trainable it must be possible to compute a gradient for back propagation). Investigate these methods and variables and freeze the weights.

You may find the `print_parameters(model)` method useful here, as it tells you which parameters of the model are trainable.

In [ ]:
# TASK - Freeze all of the weights in your model

Obviously, a network with frozen weights can't be trained, so let's make the network trainable by modifying it so that it no longer has two classification outputs, but the four that we need for CCQE, CCRes, NCQE  and NCRes.

This step is very similar to the one you performed above to permit the loading of the pretrained model, and so represents a good opportunity to refactor the code you wrote earlier into a method that can provide the required functionality for any nuumber of classes.

Aside: Jupyter notebooks are great for this kind of fast prototyping work, but they can lead to bad habits in "organising" your code into cells with lots of repetition - don't write the same code repeatedly, if something is obviously and trivially reusable, write a method (or class). You may ultimately want to factor highly generic code out into libraries in separate Python modules.

If you didn't write a function to do this earlier, a suggested method interface is given below. Note that `pass` is equivalent to a Python Todo message, replace this with your solution.

In [ ]:
# TASK - Modify the network architecture
def model_reshape(model, target_num_classes):
    """Reshapes the model

        Args:
            model: The model to be reshaped
            target_num_classes: The number of outputs required from the network
    """
    pass

You should now have a network with the correct structure, with most of the weights set by the pretrained model that we've provided for you. Before we can train this updated model, we'll set up the datasets, dataloaders and transforms required. You don't need to edit anything, we've supplied this for you because it would likely take too long to correctly implement these steps from scratch, but you may want to take a look at `data_utils.py` and `training_utils.py` to see how datasets and dataloaders are created in this case (the code is neither particularly long, nor overly complex, but there are a number of subtle details).

To briefly describe what is going on, the original ResNet18 was trained on a very particular dataset, with mandated input image size and a particular distribution of pixel values which need to be respected during transfer learning - the transforms used here ensure the input images conform to those specifications. In addition, to enhance generalisability, the images are randomly flipped about the beam axis in the training set, so the inputs in each training iteration are different.

The datasets constructed simply split the event images into separate training and validation sets and provide the standard accessor methods that allow individual images (and associated true classification) to be loaded. The dataloaders are standard PyTorch dataloaders, splitting the respective training and validation sets into batches, with images in the training set being shuffled, so the network updates its knowledge based on images presented in a different order in each epoch.

In [ ]:
set_seed(42)

batch_size = 128
num_classes = 4

transform = get_resnet_transforms()
training_set, validation_set = make_datasets(f"images", transform, valid_size=10000, train_size=8000)
dataloaders, dataset_sizes, weights = prepare_dataloaders(training_set, validation_set, batch_size, num_classes)

Now you can begin updating this model to classify neutral current events as well. Below we've provided a template training loop that you should augment to perform the training steps.

The method takes all of the arguments that we believe are required for this task, though feel free to extend or alter this to suit your needs (for example, it is not necessary to include a learning rate scheduler, but perhaps you think one might be useful).

You'll also note some summary information in the comments belowing regarding what each argument contains. Note in particular that the data loaders for the training and validation sets are accessible via the dicctionary keys 'train' and 'val' respectively.

Parts of the code with comments starting `# TASK` indicate those sections of code that require completion, but feel free to edit the surrounding code if it suits your needs.

In [ ]:
import sklearn.metrics as skm

def train_model(model, epochs, criterion, optimizer, dataloaders, dataset_sizes, device, output_filename):
    """Runs the training loop for a model

        Args:
            model: The model to be trained
            epochs: The number of epochs to train for
            criterion: The loss function to be optimised
            optimizer: The optimizer to use
            dataloaders: A dictionary containing the training ('train') and validation ('val') data loaders
            dataset_sizes: The number of samples in the training ('train') and validation ('val') datasets
            device: The device on which training should be run
            output_filename: The output model filename
        
        Returns:
            A tuple containing the best model and the metric history
    """
    # initialise a map to hold the training statistics generate throughout training
    statistics = {
        'train': { 'loss': [], 'accuracy': [], 'f1': [] },
        'val': { 'loss': [], 'accuracy': [], 'f1': [] }
    }

    epoch = 1
    learning = True
    while learning:
        for phase in ['train', 'val']:
            # TASK - Set the model to training or evaluation mode according to the current phase

            # Initialise variables used to calculate statistics for each epoch
            predictions = [None] * len(dataloaders[phase])
            truths = [None] * len(dataloaders[phase])
            running_loss = 0.0

            # iterate over the batches in the current dataloader
            for b, (inputs, labels) in enumerate(dataloaders[phase]):
                # TASK - transfer the inputs and the labels to the GPU

                # TASK reset the gradients tracked by the optimizer

                # operations on the model should compute and track gradients during training, but not during validation
                with torch.set_grad_enabled(phase == 'train'):
                    # TASK - run the network over the input images
                    # The first time you run this you may want to print the result to see
                    # if the output has the form you expect
                    
                    # TASK - compute the loss for the network classification
                    
                    _, preds = torch.max(outputs, 1)

                    # TASK - perform the back propagation step and update the optimizer
                    # Should these steps always run?

                    predictions[b] = preds
                    truths[b] = labels.data

                # Keep track of the running loss, factoring batch size - the final batch can sometimes be smaller than
                # the specified batch size, so we need to account for that to ensure a correct loss for each epoch
                running_loss += loss.item() * inputs.size(0)
            
            # calculate the loss, accuracy and f1 score for the epoch (stored in statistics)
            update_statistics(statistics[phase], running_loss / dataset_sizes[phase], predictions, truths)
            # print the statistics for the epoch
            print_statistics(statistics, phase, epoch)

        print()
        epoch += 1
        if epoch > epochs:
            learning = False

    # save the model
    save_model(model, f"{output_filename}")

    return model, statistics

The code above simply defines the steps to be taken during training and validation. Now we need to actually exectue this code.

In [ ]:
print_parameters(model)
model = model.to(device)

# TASK - create a CrossEntropyLoss loss module here, and get it to use the weights returned by prepare_dataloaders

# TASK - create an SGD optimizer module here


t0 = time.perf_counter()
# TASK - call your model training method here, you might want to try just 1 epoch to begin with
# we suggest you ultimately limit the number of epochs to 5-10

t1 = time.perf_counter()
print(f"Network {iter} trained in {t1 - t0} s")

Congratulations, you've now training a neutrino interaction classifier using transfer learning.

# Task 2: Assessing the performance

(See Slide 10)

Having trained a network, we now want to start assessing its performance in a little more detail.

You can already see a quantification of the overall network accuracy from the statistics printed at the end of each epoch (how did the network do? Are you impressed? Under-whelmed?), but this folds the performance of the different classes together and reports a single figure of merit.

What we'd like you to do now is determine per class accuracies in the validation set and also produce a *confusion matrix* (if you're not familiar with confusion matrices this is just a table presenting the number of instances of a given true class against the network classification).

Do the errors in the confusion matrix make sense?

We're not going to provide a code template here (apart from a small note below), but much of the code you will require to run network inference on the validation set and extract network classification and truth information already exists above. In addition, you may want to identify some events of interest and produce an event display to see if that helps you understand where the network might have gone wrong.

## Displaying events

With a little bit of tensor manipulation you can display an event using the matplotlib imshow function (there are other approaches, but this is a relatively simple, low effort approach. Note that when you iterate through a dataset you are getting a batch of 128 events with each iteration, in particular each batch is of the form (inputs, labels), where inputs will in turn have the shape \[128, 3, 224, 224\]. If you want to access a particular event, you can just index into the first dimension of the tensor, which will then present a tensor with the shape \[3, 224, 224\]. The first dimension here represents the U (0), V (1) and W (2) views onto the event, so if you want to display the W view of event 73 in the current batch, you can simply use

```
plt.imshow(inputs[73][2])
```

In [ ]:
# TASK - assess the network performance

# Task 3: Finetuning the network
(See Slide 11)

The first step to finetuning the network is to allow the weights that you have previously frozen to be modifiable again. Recall from earlier that you froze the weights using the `requires_grad`, depending on how general the function you wrote to perform that task was, you may be able to reuse it here.

In [ ]:
# TASK - Allow all parameters of the network to be modifiable

# You can check that the model has now frozen weights based on the output of print_parameters
print_parameters(model)

If all of your model parameters are modifiable, you can now continue training. You should be able to reuse `train_model` here (remember to change the output filename), but you may want to consider altering the learning rate of the optimiser. Try a few different values to see if it affects the behaviour - remember `train_model` saved your model from the previous stage so you can always reload it to ensure fair comparisons between runs.

In [ ]:
# TASK - continue training your saved network, comparing a few different learning rate settings for the optimizer

# Task 4: Assessing the performance
(See Slide 12)

Repeat Task 2 using the best trained network you produced from Task 3. How does the performance compare to that in Task 2?

In [ ]:
# TASK - assess the network performance